# SLR Data Pipeline

In [ ]:
# importing libraries
import pandas as pd
import requests
from tqdm import tqdm

from pybliometrics.scopus import ScopusSearch, AbstractRetrieval

import pybliometrics
pybliometrics.scopus.init()

## Search String and first search

In [ ]:
search_string = """TITLE-ABS("environmental justice" OR "environmental injustice")"""
results = ScopusSearch(search_string, verbose = True, download = True)
results.get_results_size()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
search_df = pd.DataFrame(results.results)

In [ ]:
#putting the eids or identifiers into a list for the Abstract Retrieval Step
eids = search_df["eid"].tolist()

## Importing picke - search_df

In [ ]:
import pickle

#with open('first_search.pkl', 'wb') as file:
    #pickle.dump(search_df, file)

with open('pickles/first_search.pkl', 'rb') as file:
    search_df = pickle.load(file)

#creating a copy of the search dataframe
filtered_df = search_df.copy()

## Abstract Retrieval

The code cells below are for retrieving the articles information and filter them by type and language

In [ ]:
articles = []
for eid in tqdm(eids):
    articles.append(AbstractRetrieval(eid, view='FULL'))

## Import pickle - Abstract retrieval

In [ ]:
import pickle

#with open('articles_raw_list.pkl', 'wb') as file:
#    pickle.dump(articles, file)

with open('pickles/articles_raw_list.pkl', 'rb') as file:
    articles = pickle.load(file)

## Filtering in english articles and articles and reviews, and until 2024

In [ ]:
#new columns for language and article type

filtered_df["language"] = ""
filtered_df["type"] = ""

#articles has the same length in number of rows as filter_df
#populating the columns and adding the abstract column
for i, article in enumerate(articles):
    filtered_df.loc[i,"language"] = article.language
    filtered_df.loc[i,"type"] = article.subtype
    filtered_df.loc[i,"abstract"] = article.abstract

In [ ]:
#Applying filtered selection for articles, review and english language
df = filtered_df.loc[((filtered_df["type"] == "ar") | (filtered_df["type"] == "re")) 
                    & (filtered_df["language"] == "eng"),]

df = df.reset_index(drop=True)

#creating a column with only the year
df["year"] = df.loc[:,"coverDate"].str[:4]

#excluding articles from 2025
df = df.loc[df["year"] != "2025",]
df = df.reset_index(drop=True)

#dropping articles without abstract
df_abs = df.copy()
df_abs = df.loc[df["abstract"].isna() == False,]
df_abs = df_abs.reset_index(drop=True)

In [ ]:
df_abs.info()

## Creating the dataframe for screening

In [ ]:
df_abs.columns

In [ ]:
first_df = df_abs.copy()
#selecting only the relevant columns
first_df = first_df.loc[:, ["eid", "doi", "publicationName",
                          "author_names", "year", "title", "abstract"]]

#changing the name of some columnts
first_df = first_df.rename(columns={"publicationName": "journal"})

#creating a short id for future reference
first_df = first_df.reset_index(drop=False, names = "short_id")

#filling missing values
first_df["doi"] = first_df["doi"].fillna("no doi")
first_df["author_names"] = first_df["author_names"].fillna("no names")

# Create a new column "weblink" while preserving the original "doi" column

# Create DOI Link
first_df["weblink"] = ("https://doi.org/" + first_df["doi"]).where(first_df["doi"] != "no doi", "no link")

# Create Google Scholar Link
first_df["scholar_link"] = ("https://scholar.google.com/scholar?q=" + first_df["doi"]).where(first_df["doi"] != "no doi", "no link")

In [ ]:
#reordering the columns

weblink_col = first_df.pop("weblink")
scholar_link_col = first_df.pop("scholar_link")

# Find the position of "doi"
doi_index = first_df.columns.get_loc("doi")

# Insert the "weblink" column right after "doi"
first_df.insert(doi_index + 1, "weblink", weblink_col)

# Insert the "scholar_link" column right after "weblink"
first_df.insert(doi_index + 2, "scholar_link", scholar_link_col)

### Shuffling the dataframe

In [ ]:
shuffled_df = first_df.sample(frac=1).reset_index(drop=True)

shuffled_df["included"] = "x"

### Loading the pickle with the shuffled dataframe

In [ ]:
import pickle
#with open('pickles/shuffled_df.pkl', 'wb') as file:
    #pickle.dump(shuffled_df, file)

with open('pickles/shuffled_df.pkl', 'rb') as file:
    shuffled_df = pickle.load(file)

In [ ]:
shuffled_df["included"] = "-"
shuffled_df["article_type"] = "-"

### Importing last edited table

This happens when some coders have run a test on the table on google spreadsheets. But to modify the whole table again, without losing th first input from coders, I need to integrate this into the original shuffled_df

In [ ]:
spreadsheet_id = '1Ai8o0my3obDICNLMUFKcCYv0Q6Od9NWc'  # Replace with your Google Sheet ID
sheets_names = ['EF', 'CG', 'HVW', 'GR']  # Replace with your sheet name

imported_dfs = []
for sheet_name in sheets_names:
    csv_url = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    imported_dfs.append(pd.read_csv(csv_url))

### Getting the data already inserted

I am retrieving the data that the coders put already and saving it as lists

In [ ]:
EF_coded = list(imported_dfs[0].iloc[:99, -1])
CG_coded = list(imported_dfs[1].iloc[:99, -1])
HVW_coded = list(imported_dfs[2].iloc[:50, -1])
GR_coded = list(imported_dfs[3].iloc[:100, -1])

### Distribution of the abstract screening table

Here is the distribution of articles per coders for the abstract screening process

In [ ]:
distribution = {
    "Elli": ["EF", 800],
    "Charlotte": ["CG", 800],
    "Henrik": ["HVW", 300],
    "Gustavo": ["GR", 1000],
    "Wanja": ["WT", 100],
    "Joanna": ["JK", 100],
    "Victor": ["VC", 100],
    "Max": ["MFK", 100],
    "Kristina": ["KBB", 400],
    "Clara": ["CUW", 200],
    "Antonia": ["AU", 100],
    "Dagmar": ["DBM", 400],
    "Madawi": ["MN", 100],
    "Emre": ["EO", 100],
    "Luana": ["LK", 100],
    "Jaqueline": ["JL", 100],
    "Polyana": ["PL", 150],
    "Extra3": ["EX3", 100],
    "Extra4": ["EX4", 100],
    "Extra5": ["EX5", 100],
    "Extra6": ["EX6", 100],
    "Extra7": ["EX7", 100],
    "Extra8": ["EX8", 100],
    "Extra9": ["EX9", 126]
}

### Creating different dataframes for each coder

In [ ]:
#this piece of code is to divide automatically the dataframe according to the distribution
dfs = []
start_index= 0
end_index = 0
for key, value in distribution.items():
    num = value[1] # contains the number of abstracts to be screened
    end_index = end_index + num 
    table = shuffled_df.iloc[start_index:end_index,:]
    table2 = table.copy()
    dfs.append(table2) 
    distribution[key].append(table)
    start_index = start_index + num

### Updating the respective dataframes

In [ ]:
distribution["Elli"][2].iloc[:99,-2] = EF_coded
distribution["Charlotte"][2].iloc[:99,-2] = CG_coded
distribution["Henrik"][2].iloc[:50,-2] = HVW_coded
distribution["Gustavo"][2].iloc[:100,-2] = GR_coded

#changing these columns to string

distribution["Elli"][2].iloc[:,-2] = distribution["Elli"][2].iloc[:,-2].astype(str)
distribution["Charlotte"][2].iloc[:,-2] = distribution["Charlotte"][2].iloc[:,-2].astype(str)
distribution["Henrik"][2].iloc[:,-2] = distribution["Henrik"][2].iloc[:,-2].astype(str)
distribution["Gustavo"][2].iloc[:,-2] = distribution["Gustavo"][2].iloc[:,-2].astype(str)

distribution["Elli"][2].loc[distribution["Elli"][2]["included"] == "1.0", ["included"]] = "yes"
distribution["Elli"][2].loc[distribution["Elli"][2]["included"] == "0.0", ["included"]] = "no"
distribution["Charlotte"][2].loc[distribution["Charlotte"][2]["included"] == "1.0", ["included"]] = "yes"
distribution["Charlotte"][2].loc[distribution["Charlotte"][2]["included"] == "0.0", ["included"]] = "no"
distribution["Henrik"][2].loc[distribution["Henrik"][2]["included"] == "1", ["included"]] = "yes"
distribution["Henrik"][2].loc[distribution["Henrik"][2]["included"] == "0", ["included"]] = "no"
distribution["Gustavo"][2].loc[distribution["Gustavo"][2]["included"] == "1.0", ["included"]] = "yes"
distribution["Gustavo"][2].loc[distribution["Gustavo"][2]["included"] == "0.0", ["included"]] = "no"


### Putting the test input in the respective coders' dataframes

In [ ]:
distribution["Charlotte"][2].head(10)

# Creating Excel File with highlighting

## Function for coloring strings

In [ ]:
import re

# to search the regular expression in the string
pattern = re.compile(r'(environmental injustice|environmental justice)',re.IGNORECASE)

def highlight_keywords(worksheet, row, col, text):
    """
    Splits 'text' around each regex match (pattern)
    and writes partial substrings using XlsxWriter's write_rich_string().
    Matches for 'environmental injustice' or 'environmental justice' are in red.
    Everything else is in black.
    """
    # Find all matches + the segments in between
    matches = list(pattern.finditer(text))
    
    # If no matches, write the entire text in black
    if not matches:
        worksheet.write(row, col, text, black_format)
        return
    
    # Build a list of alternating segments with their formats
    rich_segments = []
    last_end = 0
    
    for match in matches:
        start, end = match.span()
        
        # Add text before the match, in black
        if start > last_end:
            segment_before = text[last_end:start]
            rich_segments.append(black_format)
            rich_segments.append(segment_before)
        
        # Add the matched substring, in red
        match_str = text[start:end]
        rich_segments.append(red_format)
        rich_segments.append(match_str)
        
        last_end = end
    
    # Add any remaining text after the last match, in black
    if last_end < len(text):
        segment_after = text[last_end:]
        rich_segments.append(black_format)
        rich_segments.append(segment_after)
    
    # Check if there are fewer than two format/text pairs
    if len(rich_segments) <= 2:
        # If there's only one fragment, fall back to normal write()
        # Default to black or red format depending on the first match
        worksheet.write(row, col, text, red_format if matches else black_format)
    else:
        # Use write_rich_string for multiple fragments
        worksheet.write_rich_string(row, col, *rich_segments)

## Creating Excel writer using xlsxwriter

In [ ]:
import xlsxwriter

output_file = "pilot2.xlsx"
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')

# Add a blank workbook & worksheet
workbook  = writer.book

# Define formats for highlighting
red_format = workbook.add_format({'font_color': '#DA70D6', 'bold': True, 'valign': 'vcenter'})
black_format = workbook.add_format({'font_color': 'black', 'text_wrap': True, 'valign': 'vcenter'})
header_format = workbook.add_format({'font_color': 'white', 'bold': True, 'align': 'center', 'bg_color': 'green'})
align_format = workbook.add_format({'text_wrap': True, 'valign': 'vcenter'}) 

for key,val in distribution.items():
    person_id = val[0]  #the person id
    df = val[-1]      #the respective dataframe
    nrows = val[1] + 1 # the number of rows fo the dataframe
    worksheet = workbook.add_worksheet(person_id)



    dropdown_options = ['yes', 'no', '-']
    category_col_letter = 'K'  # Assuming "category" is in column K
    worksheet.data_validation(f'{category_col_letter}2:{category_col_letter+str(nrows)}', {
    'validate': 'list',
    'source': dropdown_options,
    'input_message': 'Select value',
    'error_message': 'Invalid selection! Please choose from the dropdown.'
    })

    dropdown_options2 = ['empirical', 'conceptual', 'review', 'unknown', '-']
    category_col_letter2 = 'L'  # Assuming "category" is in column K
    worksheet.data_validation(f'{category_col_letter2}2:{category_col_letter2+str(nrows)}', {
    'validate': 'list',
    'source': dropdown_options2,
    'input_message': 'Select value',
    'error_message': 'Invalid selection! Please choose from the dropdown.'
    })
    
    # writing header row
    for col_idx, col_name in enumerate(df.columns):
        worksheet.write(0, col_idx, col_name, header_format)

    # Write data rows
    for row_idx in range(len(df)):
        for col_idx, value in enumerate(df.iloc[row_idx]):
            col_name = df.columns[col_idx]
            
            if col_name in ["title", "abstract"] and isinstance(value, str):
                # Apply partial substring highlighting
                highlight_keywords(worksheet, row_idx + 1, col_idx, value)

                char_per_line = 50  # Adjust based on actual column width
                num_lines = -(-len(value) // char_per_line)  # Ceiling division for line count
                row_height = max(7 * num_lines, 7)
                worksheet.set_row(row_idx + 1, row_height)  # Set row height

            elif col_name in ["included", "article_type"]:
            # Write category values normally (dropdown will apply to this column)
                worksheet.write(row_idx + 1, col_idx, value, black_format)

            else:
                # Write other columns normally
                worksheet.write(row_idx + 1, col_idx, value, black_format)

    #adjusting size of columns
    worksheet.set_column('E:E', 15)
    worksheet.set_column('G:G', 15)  
    worksheet.set_column('I:I', 50, align_format)  
    worksheet.set_column('J:J', 100, align_format)
    worksheet.set_column('K:K', 10) 
    worksheet.set_column('L:L', 17) 

# Close (this actually writes the file)
workbook.close()

# Uploading to google sheets

In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import gspread

# Define the required scopes
scopes = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets"
]

# Authenticate using the credentials file
flow = InstalledAppFlow.from_client_secrets_file('google_credentials.json', scopes=scopes)
credentials = flow.run_local_server(port=0)

# Initialize Google Drive and Google Sheets API clients
drive_service = build('drive', 'v3', credentials=credentials)
gc = gspread.authorize(credentials)

### Creating Folder

In [ ]:
# Create a folder in Google Drive
folder_metadata = {
    'name': 'Environmental Justice - SLR',  # Folder name
    'mimeType': 'application/vnd.google-apps.folder'
}

folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
folder_id = folder.get('id')
print(f"Folder created successfully! Folder ID: {folder_id}")

### Creating Spreadsheet

In [ ]:
# Create a Google Spreadsheet in the folder
spreadsheet_metadata = {
    'name': 'abstract_screening',  # Spreadsheet name
    'mimeType': 'application/vnd.google-apps.spreadsheet',
    'parents': [folder_id]  # Place in the created folder
}

spreadsheet = drive_service.files().create(body=spreadsheet_metadata, fields='id,webViewLink').execute()
spreadsheet_id = spreadsheet.get('id')
spreadsheet_url = spreadsheet.get('webViewLink')

print(f"Spreadsheet created successfully! URL: {spreadsheet_url}")

### Updating spreadsheet

In [ ]:
dataframes = {"Ellie": elli_df, 
              "Charlotte": charlotte_df, 
              "Gustavo": gustavo_df}

spreadsheet = gc.open_by_key(spreadsheet_id)

for sheet_name, df in dataframes.items():
    # Create a new worksheet or open if it exists
    try:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(df.shape[0]+10), cols=str(df.shape[1]+10))
    except gspread.exceptions.APIError:
        worksheet = spreadsheet.worksheet(sheet_name)
    
    # Write data to the worksheet
    data = [df.columns.values.tolist()] + df.values.tolist()  # Convert DataFrame to list of lists
    worksheet.update(data)

print("DataFrames uploaded successfully!")

### Deleting sheets

In [ ]:
# Open the spreadsheet by its ID
spreadsheet = gc.open_by_key(spreadsheet_id)

# Find and delete "Sheet1"
try:
    worksheet = spreadsheet.worksheet("Sheet1")
    spreadsheet.del_worksheet(worksheet)
    print("Default 'Sheet1' deleted successfully!")
except gspread.exceptions.WorksheetNotFound:
    print("'Sheet1' does not exist or has already been deleted.")

### Changing permissions

In [ ]:
# Share the file with "Anyone with the link" and set to "Editor"
drive_service.permissions().create(
    fileId=spreadsheet_id,  # The ID of your spreadsheet
    body={
        'type': 'anyone',  # Share with anyone
        'role': 'writer'   # Grant editing permissions
    },
    fields='id'
).execute()

print("The spreadsheet is now editable by anyone with the link!")

### Creating Log Spreadsheet

#### Authentication

In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Define the scopes for Google Drive and Sheets
scopes = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets"
]

# Authenticate using OAuth 2.0
flow = InstalledAppFlow.from_client_secrets_file(
    'google_credentials.json',  # Path to your OAuth credentials file
    scopes=scopes
)
credentials = flow.run_local_server(port=0)

# Initialize the Drive API
drive_service = build('drive', 'v3', credentials=credentials)

In [ ]:
folder_id = "1jQgEHv8m7ZtXeBoiQtp2e61i9FB-7Dpc"

# Create a new Google Sheets file in the folder
spreadsheet_metadata = {
    'name': 'Change Log',  # Name of the new spreadsheet
    'mimeType': 'application/vnd.google-apps.spreadsheet',
    'parents': [folder_id]  # Place it inside the folder
}

spreadsheet = drive_service.files().create(
    body=spreadsheet_metadata,
    fields='id,webViewLink'
).execute()

# Retrieve the spreadsheet ID and URL
spreadsheet_id = spreadsheet.get('id')
spreadsheet_url = spreadsheet.get('webViewLink')

print(f"Change Log spreadsheet created successfully: {spreadsheet_url}")

## Uploading excel workbook

In [ ]:
from gspread_dataframe import set_with_dataframe

# Path to the Excel file
excel_file = "pilot.xlsx"

# Load the Excel file with pandas
excel_data = pd.read_excel(excel_file, sheet_name=None)  # Load all sheets as a dictionary

folder_id = "1jQgEHv8m7ZtXeBoiQtp2e61i9FB-7Dpc"

# Create a Google Spreadsheet in the folder
spreadsheet_metadata = {
    'name': 'abstract_screening',  # Spreadsheet name
    'mimeType': 'application/vnd.google-apps.spreadsheet',
    'parents': [folder_id]  # Place in the created folder
}

spreadsheet = drive_service.files().create(body=spreadsheet_metadata, fields='id,webViewLink').execute()
spreadsheet_id = spreadsheet.get('id')
spreadsheet_url = spreadsheet.get('webViewLink')

print(f"Spreadsheet created successfully! URL: {spreadsheet_url}")

spreadsheet = gc.open_by_key(spreadsheet_id)

# Iterate over sheets in the Excel file and upload each sheet
for sheet_name, sheet_data in excel_data.items():
    # Add a new worksheet for each sheet
    worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=str(sheet_data.shape[0] + 1), cols=str(sheet_data.shape[1]))
    
    # Upload the data from the Excel sheet to the Google Sheet
    set_with_dataframe(worksheet, sheet_data)

# Remove the default worksheet (if empty)
default_sheet = spreadsheet.sheet1
spreadsheet.del_worksheet(default_sheet)

print(f"Excel workbook uploaded successfully to Google Sheets: {spreadsheet.url}")

# Share the file with "Anyone with the link" and set to "Editor"
drive_service.permissions().create(
    fileId=spreadsheet_id,  # The ID of your spreadsheet
    body={
        'type': 'anyone',  # Share with anyone
        'role': 'writer'   # Grant editing permissions
    },
    fields='id'
).execute()

print("The spreadsheet is now editable by anyone with the link!")